# 🎛️ Controlador Central - Hackathon Churn Project

Este notebook atua como **orquestrador mestre** de todo o projeto. Ele é capaz de subir a infraestrutura, iniciar a API Java, executar testes de carga e desligar tudo ao final.

> **Status Atual:** Híbrido (Java Spring Boot + Python AI Service + MongoDB)

## 1. Preparação e Infraestrutura (Docker)
Garante que o banco de dados e o serviço de IA (containers) estejam rodando.

In [ ]:
import subprocess
import time
import requests
import os
import pandas as pd
import signal

# Função auxiliar para rodar comandos de shell e imprimir saída
def run_shell(command):
    process = subprocess.run(command, shell=True, capture_output=True, text=True)
    if process.returncode == 0:
        print(f"✅ {command.split()[0]} OK")
        if process.stdout: print(process.stdout[:200] + "...") # Mostra inicio do log
    else:
        print(f"❌ Erro em {command}")
        print(process.stderr)

print("🐋 Reiniciando Containers...")
run_shell("docker-compose down")
run_shell("docker-compose up -d")
print("⏳ Aguardando 5s para estabilização do Mongo...")
time.sleep(5)

## 2. Iniciando a API Java (Background)
Aqui iniciamos o servidor Spring Boot através do Maven em um processo separado, para não travar o Notebook.

In [ ]:
# Caminho para o executável do Maven wrapper no projeto
mvn_cmd = ".\\apache-maven-3.9.6\\bin\\mvn.cmd spring-boot:run"

print(f"🚀 Iniciando API Java em Background...\nComando: {mvn_cmd}")

# Inicia processo independente
api_process = subprocess.Popen(
    mvn_cmd, 
    shell=True, 
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE
)

print(f"✅ Processo iniciado (PID: {api_process.pid})")
print("⏳ Aguardando inicialização (aprox. 15-30s)...")

## 3. Healthcheck (Aguardando API ficar Online)
Loop que verifica se a API já respondeu no endpoint `/api/health`.

In [ ]:
url_health = "http://localhost:9999/api/health"
max_retries = 30
api_ready = False

for i in range(max_retries):
    try:
        r = requests.get(url_health, timeout=2)
        if r.status_code == 200:
            print(f"\n✅ API ONLINE! ({r.json()})")
            api_ready = True
            break
    except:
        print(".", end="")
        time.sleep(2)

if not api_ready:
    print("\n❌ Tempo limite excedido. Verifique se o Java startou corretamente.")
    # Opcional: imprimir logs de erro
    # print(api_process.stdout.read().decode('utf-8'))

## 4. Executando Pipeline de Testes
Agora que a API está de pé, rodamos o fluxo de registro e carga.

In [ ]:
if api_ready:
    # 1. Autenticação (Criar Token)
    # (Simplificado, assumindo usuario criado no boot ou criando agora)
    try:
        requests.post("http://localhost:9999/usuarios", json={"login": "admin_nb", "senha": "123"})
    except: pass
    
    token_resp = requests.post("http://localhost:9999/login", json={"login": "admin_nb", "senha": "123"})
    token = token_resp.json()['token']
    print("🔑 Token JWT obtido:", token[:15] + "...")
    
    # 2. Enviar Lote de Arquivo
    file_path = "teste_batch.csv"
    url_batch = "http://localhost:9999/api/churn/batch/optimized"
    
    print(f"📤 Processando {file_path}...")
    with open(file_path, 'rb') as f:
        files = {'file': (file_path, f, 'text/csv')}
        headers = {'Authorization': f'Bearer {token}'}
        resp = requests.post(url_batch, files=files, headers=headers)
        
    if resp.status_code == 200:
        print("✅ Lote processado com sucesso!")
        # Carregar resultado para visualizar
        from io import BytesIO
        df = pd.read_csv(BytesIO(resp.content))
        display(df.head())
    else:
        print("❌ Erro no batch execution:", resp.text)

## 5. Encerrando o Projeto
Importante: Matar o processo Java para liberar a porta 9999 e parar os containers.

In [ ]:
print("🛑 Parando API Java...")
# No Windows, matar subprocesso shell nem sempre mata os filhos.
# O comando taskkill é mais garantido para matar a árvore de processos do Java que iniciamos.
subprocess.run("taskkill /F /IM java.exe", shell=True)

print("🛑 Parando Containers...")
run_shell("docker-compose stop")

print("👋 Sistema desligado.")